# Data Science I - Projeto final

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline

In [ ]:
df = pd.read_csv('titanic-data-6.csv')

---

# Preparar dados (Wrangle data)

Consultar o formato do dataframe, número de linhas e colunas.

In [ ]:
df.shape

Consultar as primeiro cinco linhas do dataframe.

In [ ]:
df.head()

Consultar informações como número de entradas, colunas e seus tipos de dados.

In [ ]:
df.info()

Renomear as colunas

In [ ]:
df.rename(
    columns={
        'PassengerId': 'id_passageiro', 
        'Survived': 'sobreviveu', 
        'Pclass':'classe', 
        'Name':'nome', 
        'Sex':'sexo', 
        'Age':'idade', 
        'SibSp':'irmao_conjuge', 
        'Parch':'pais_filhos', 
        'Ticket':'ticket', 
        'Fare':'tarifa', 
        'Cabin':'cabine',
        'Embarked': 'embarcou'
    }, inplace=True)

Verificar a existência de linhas duplicadas.

In [ ]:
df.duplicated().sum()

Verificar se há nomes repetidos

In [ ]:
df['nome'].duplicated().sum()

Verificar variedade de registros únicos em sexo

In [ ]:
df['sexo'].nunique()

In [ ]:
df['sexo'].unique()

Tradução de valores de 'sexo'

In [ ]:
pt_sex = {'male': 'Masculino', 'female':'Feminino'}
df['sexo'] = df['sexo'].replace(pt_sex.keys(), pt_sex.values())

In [ ]:
df['sexo'].unique()

Verificar variedade de registros únicos em sobreviventes

In [ ]:
df['sobreviveu'].nunique()

In [ ]:
df['sobreviveu'].unique()

Substituição dos valores de sobreviventes de 0, 1 para 'Não' e 'Sim', respectivamente

In [ ]:
pt_survived = {'0':'Não', '1':'Sim'}
df['sobreviveu'] = df['sobreviveu'].astype(str).replace(pt_survived.keys(), pt_survived.values())

In [ ]:
df['sobreviveu'].head()

Verificar se existe valores nulos

In [ ]:
df.isnull().any()

Foram encontrados valores nulos nas colunas 'idade', 'cabine' e 'embarcou'

Preenchimento de valores nulos em idade com seu valor médio

In [ ]:
df['idade'].fillna(round(df['idade'].mean()), inplace=True)

Verificar os registros nulos em local de embarque (Embarked)

In [ ]:
df['embarcou'].isnull().sum()

In [ ]:
df[df['embarcou'].isnull()]

Verificar se existe passageiros sem local de embarque marcados como não sobreviventes

In [ ]:
df[df['sobreviveu'] == 'Não']['embarcou'].isnull().any().sum()

Por não existir passageiros sem local de embarque marcado como não sobrevivente todos serão removidos, considerando que estes não estavam a bordo

In [ ]:
df.dropna(subset=['embarcou'], inplace=True)

Validar a remoção de passageiros sem local de embarque

In [ ]:
df['embarcou'].isnull().any().sum()

In [ ]:
df.shape

Listar os valores únicos para a coluna referente ao local em que o passageiro embarcou

In [ ]:
df['embarcou'].unique()

Substituição do nome dos locais de embarque abreviados pelo nome completo

In [ ]:
embarked_places = {'S': 'Southampton', 'C': 'Cherbourg', 'Q': 'Queenstown'}
df['embarcou'].replace(embarked_places.keys(), embarked_places.values(), inplace=True)

Categorização das idades dos passageiros

In [ ]:
ages = [0, 15, 25, 65, np.inf]
ages_categories = ['Criança', 'Jovem', 'Adulto', 'Idoso']

df['categoria_idade'] = pd.cut(df['idade'], bins=ages, labels=ages_categories,right=True)

Validar o resultado da classificação de idade

In [ ]:
df['categoria_idade'].unique()

---

# Explorar (Explore)

Matriz de gráficos de dispersão para ideias

In [ ]:
pd.plotting.scatter_matrix(df[['sobreviveu', 'classe', 'idade', 'irmao_conjuge', 'pais_filhos', 'tarifa']], figsize=(15,15));

In [ ]:
df.head()

---

# Configurações e funções

Definir estilo padrão para os gráficos com Seaborn

In [ ]:
sns.set_style('whitegrid')

Definição de função para criar plot figure com título

In [ ]:
def create_figure(title):
    """
        Criar uma figure com titulo posicionado
        
        Args:
            title (String): Valor a ser preenchido no título do gráfico 
            
        Return:
            (plt.Figure) Retorna uma Figure para adicionar gráficos (Plots)
    """
    fig = plt.figure(title)
    fig.suptitle(title, fontsize='x-large', horizontalalignment='center', verticalalignment='bottom')
    plt.subplots_adjust(top=.9)
    plt.tight_layout()
    fig.set_figwidth(20)
    
    return fig


---

# Perguntas

## 1. Quantos sobreviveram?

Tabela de contagem e proporção de sobreviventes e mortes

In [ ]:
def get_survivors():
    """
        Exibir tabela com contagem de sobreviventes e proporção
        
        Return (dataframe)
    """
    # Definição do dataframe de proporções
    df_survivors = df['sobreviveu'].value_counts().reset_index()
    df_survivors.rename(columns={'index': 'sobreviveu', 'sobreviveu': 'total'}, inplace=True)
    df_survivors['proporcao'] = round(df_survivors['total'].apply(lambda value: value / df_survivors['total'].sum()) * 100, 2)
    
    return df_survivors

In [ ]:
df_survivors = get_survivors()
df_survivors[df_survivors['sobreviveu'] == 'Sim']

Definição de função para gráfico de pie para a proporção de sobreviventes

In [ ]:
def plot_survivors():
    """
        Exibir pie plot de sobreviventes
    """
    df_survivors = get_survivors()
    fig = create_figure('')
    
    # pie plot
    ax_pie = fig.add_subplot(121)
    
    values = df_survivors['proporcao'] * 100
    explode = np.full(df_survivors.shape[0], 0.05)
    labels = df_survivors['sobreviveu'].replace(['Sim','Não'],['Sobreviveu', 'Não sobreviveu'])

    ax_pie.axis('equal')
    ax_pie.pie(
        values, 
        explode=explode, 
        labels=labels, 
        autopct='%1.1f%%', 
        shadow=True,
        startangle=90,
    );
    
    ax_pie.set_title('SOBREVIVENTES')
    
    # table plot
    ax_table = fig.add_subplot(122)
    table = ax_table.table(
        cellText = df_survivors.values,
        colLabels = ['SOBREVIVEU', 'TOTAL', 'PROPORÇÃO'],
        loc='center',        
    )
    ax_table.axis('off')
    ax_table.set_title('TABELA DE SOBREVIVENTES')
    table.auto_set_font_size(False)
    table.set_fontsize(13)
    
plot_survivors()

## 2. Quantos morreram?

Seleção dos que não sobreviveram no dataframe criado na questão anterior

In [ ]:
df_survivors[df_survivors['sobreviveu'] == 'Não']

## 3. Quantas crianças sobreviveram?

In [ ]:
df.query('categoria_idade == "Criança"')['sobreviveu'].value_counts()

In [ ]:
df.query('categoria_idade == "Criança"')['sobreviveu'].value_counts(normalize=True)

49 crianças sobreviveram

## 4. Quantas crianças morreram?

In [ ]:
df.query('categoria_idade == "Criança"')['sobreviveu'].value_counts()

34 crianças morreram

Definição de função para criar gráfico de sobreviventes classificados como crianças 

In [ ]:
def plot_children_balance():
    """
        Gráfico de crianças sobreviventes e não sobreviventes
    """
    df_children = df.query('categoria_idade == "Criança"')['sobreviveu'].value_counts(normalize=True).to_frame().reset_index()
    df_children.rename(columns={'index': 'sobreviveu', 'sobreviveu': 'percentual'}, inplace=True)
    df_children['percentual'] = round(df_children['percentual'] * 100, 2)
    
    ax = sns.barplot(
        data = df_children,
        x = 'sobreviveu',
        y = 'percentual',
        palette = 'Blues_d',
    )
    
    plt.title('SOBREVIVENTES: CRIANÇAS')
    plt.xlabel('Sobreviveu')
    plt.ylabel('Percentual')
    
    
plot_children_balance()


## 5. Qual a idade média entre os sobreviventes?

Seleção dos sobreviventes e cáculo de idade média entre eles

In [ ]:
df[df['sobreviveu'] == 'Sim']['idade'].mean()

Definição da função para imprimir a distribuição e média de idade

In [ ]:
def plot_age_dist():
    """
        Gráfico de distribuição de idade
    """
    fig = create_figure('DISTRIBUIÇÃO DE IDADE')
    ax = fig.add_subplot(111)
    df_ages_surv = df[df['sobreviveu'] == 'Sim']['idade']
    df_ages_dead = df[df['sobreviveu'] == 'Não']['idade']
    
    sns.kdeplot(df_ages_surv, shade=True, label= 'Sobreviveram',)
    sns.kdeplot(df_ages_dead, shade=True, label='Não sobreviveram',)
    
    ax.set_xlabel('Idade')
    ax.set_ylabel('Densidade')
    
    
plot_age_dist()

## 6. Qual a idade média entre os que não sobreviveram?

In [ ]:
round(df.query('sobreviveu == "Não"')['idade'].mean(),2)

## 7. Qual a relação dos sobreviventes com a classe de ingresso?

Pesquisa e normalização de sobreviventes por classe

In [ ]:
df.groupby('classe')['sobreviveu'].value_counts(normalize=True)

In [ ]:
df.groupby('classe')['sobreviveu'].value_counts()

In [ ]:
def plot_surv_by_class():
    """
        Gráfico de sobreviventes por classe
    """
    df_surv_by_class = df.groupby('classe')['sobreviveu'].value_counts(normalize=True).to_frame()
    df_surv_by_class.rename(columns={'sobreviveu':'percentual'}, inplace=True)
    df_surv_by_class.reset_index(inplace=True)
    df_surv_by_class['percentual'] = round(df_surv_by_class['percentual'] * 100, 2)

    ax = sns.barplot(
        data = df_surv_by_class,
        x = 'classe',
        y = 'percentual',
        hue = 'sobreviveu',
        palette = 'Blues_d',
    )

    plt.title('SOBREVIVENTES POR CLASSE')
    plt.xlabel('Classe')
    plt.ylabel('Percentual');
    
    return df_surv_by_class
    
plot_surv_by_class()

A classe que apresentou maior número de sobreviventes foi a primeira com 62,6%.

## 8. Qual o sexo mais relevante entre os sobreviventes?

In [ ]:
def survivors_by_sex():
    pass

df_surv_by_sex = df.query('sobreviveu == "Sim"')['sexo'].value_counts().to_frame().reset_index()


In [ ]:
df.query('sobreviveu == "Sim"')['sexo'].value_counts(normalize=True).reset_index()

In [ ]:
df.query('sobreviveu == "Sim" & sexo == "Feminino"')['classe'].value_counts()

O gênero mais presente entre os sobreviventes é o feminino

In [ ]:
def sex_survivors_compare():
    """
        Exibir gráfico de comparação de sobreviventes do sexo masculino e feminino
    """
    columns = ['sexo', 'proporcao']
    df_surv = df.query('sobreviveu == "Sim"')['sexo'].value_counts(normalize=True).reset_index()
    df_surv.columns = columns
    df_surv['proporcao'] = round(df_surv['proporcao'] * 100, 2)
    
    ax = sns.factorplot(
        data = df_surv,
        x = 'sexo',
        y = 'proporcao',
        kind = 'bar',
        palette = 'Blues_d',
    );
    
    plt.title('SOBREVIVENTES: CLASSIFICADOS POR SEXO')
    plt.ylabel('Percentual')
    plt.xlabel('Sobreviventes')
    plt.tight_layout()

sex_survivors_compare()

## 9. Qual a relação dos sobreviventes com a classe e sexo?

In [ ]:
df.groupby(['classe','sexo'])['sobreviveu'].value_counts(normalize=True)

As mulheres da primeira classe representaram a maior taxa de sobrevivência, 96,7% e a menor taxa de sobreviventes foi dos homens da terceira classe com apenas 13,5% de sobreviventes.

In [ ]:
def plot_survivors_by_class_sex():
    """
        Exibir gráfico de sobreviventes por sexo e classe
    """
    df_surv = df.groupby(['classe','sexo'])['sobreviveu'].value_counts(normalize=True).to_frame()
    df_surv.columns = df_surv.columns.get_level_values(0)
    df_surv.rename(columns={'sobreviveu': 'percentual'}, inplace=True)
    df_surv['percentual'] = round(df_surv['percentual'] * 100, 2)
    df_surv.reset_index(inplace=True)
    
    ax = sns.factorplot(
        data = df_surv.query('sobreviveu == "Sim"'),
        x = 'classe',
        y = 'percentual',
        hue = 'sexo',
        kind = 'bar',
        palette = 'Blues_d',
    );
    
    plt.title('SOBREVIVENTES: CLASSIFICADOS POR SEXO E CLASSE')
    plt.ylabel('Percentual')
    plt.xlabel('Classe')
    plt.tight_layout() 
    
    return df_surv.query('sobreviveu == "Sim"')
    
plot_survivors_by_class_sex()


## 10. Qual ponto de embarque recebeu mais passageiros?

Calcular o volume de embarque para cada local.

In [ ]:
df['embarcou'].value_counts()

Exibir o ponto com maior frequência

In [ ]:
df['embarcou'].describe()

O ponto de embarque que recebeu mais passageiros foi Southampton

## 11. Qual a relação entre as tarifas e os sobreviventes?

In [ ]:
df[df['sobreviveu'] == 'Sim']['tarifa'].describe()

In [ ]:
df[df['sobreviveu'] == 'Não']['tarifa'].describe()

Os sobreviventes apresentam uma tarifa média superior aos que não sobreviveram.

## 12. Quantos bilhetes únicos existem?

In [ ]:
df['ticket'].nunique()

## 13. Quantos bilhetes duplicados existem?

In [ ]:
df['ticket'].duplicated().sum()

## 14. Quais as características dos passageiros com bilhetes duplicados?

In [ ]:
df_duplicated = df[df['ticket'].duplicated(keep=False)]

In [ ]:
df_duplicated.describe(include='all')

Pela descrição dos dados de tickets duplicados é possivel constatar que:
- são no total 342 tickets duplicados,
- destes 342, 177 sobreviveram
- com maior frequência na terceira classe,
- 178 são do sexo feminino
- o ticket identificado como 1601 foi registrado sete vezes
- a cabine mais frequente foi a G6, registrada quatro vezes
- 241 embarcaram em Southampton
- 199 são adultos
- idade média de 26,9 anos

## 15. Qual a relação entre os sobreviventes e os pontos de embarque?

Seleção dos dados de passageiros agrupados por local

In [ ]:
df_surv_embark = df.groupby(['embarcou'])

Relação de sobreviventes por local de embarque

In [ ]:
df_surv_embark['sobreviveu'].value_counts(normalize=True)

Contagem de sobreviventes por local de embarque

In [ ]:
df_surv_embark['sobreviveu'].value_counts()

Contagem de embarques por local

In [ ]:
df_surv_embark['classe'].count()

Proporção de embarques por local

In [ ]:
df['embarcou'].value_counts(normalize=True)

O local de embarque que apresentou a maior número de sobreviventes foi Southampton com 217 (33,6%) sobreviventes do total de 644 que embarcaram, entretanto, Cherbourg apresentou maior percentual de sobreviventes 55,3%, 93 passageiros do total de 168. Queenstown apresentou 38,9% (30) de sobreviventes do total de 77 embarques.

---

# Conclusão

Ao analisar a amostra de dados de passageiros do Titanic, foi possível chegar as seguintes conclusões.
- A primeira classe apresentou o maior percentual de sobreviventes, com 62,6%, contra 47,2% da segunda e 24,2% da terceira. Tal diferença pode apontar que houve alguma facilidade ou favorecimento no acesso aos botes salva vidas para os passageiros da primeira classe.
- Dentre os sobreviventes, 68% são mulheres e 32% homens
- O grupo de classe e sexo que apresentou maior taxa de sobreviventes foi o feminino da primeira classe, com a relação de 96,7%
- 59,0% das crianças a bordo foram salvas
- A idade média entre os sobreviventes era de 28,4 anos.
- O embarque de passageiros de Southampton representou 72,4%

# Limitações

Inicialmente ao avaliar o conjunto de dados foi detectado que haviam campos como idade, embarcou e cabine, não preenchidos e que potencialmente prejudicariam a análise. 
Assim, foram feitos alguns ajustes como:
- Campos de idade com valor nulo foram preenchidos com a média
- Foi efetuada uma breve análise sobre os passageiros que apresentavam o campo 'embarcou' como nulo e se haviam sobrevivido, assim, dado que o retorno foi positivo, estes foram removidos do conjunto de dados considerando que estes não embarcaram.

Valores abreviados em local de embarque (embarcou) foram substituídos pelo nome do local sem abreviação.

Foram aplicadas traduções nos nomes das colunas e nos valores dos campos sobreviveu e sexo para auxiliar na criação de legendas dos gráficos.

Foi criado um campo de categorização da idade dos passageiros com o objetivo de mapea-los facilmente e levantar informações a respeito de cada grupo.

---

# Apresentação

Mesmo após um século de seu naufrágio, que ocorreu em 1912, o Titanic é considerado um dos maiores desastres marítimos em tempos de paz. Sua história rendeu livros, filmes e diversos documentarios que buscam explorar e levar informações e curiosidades a seu respeito ao público. Nesse mesmo sentido, esse projeto tem como objetivo explorar o conjunto de dados do Titanic e tentar responder uma série de perguntas pertinentes. O arquivo está disponível no formato CSV através do [link](https://d17h27t6h515a5.cloudfront.net/topher/2017/October/59e4fe3d_titanic-data-6/titanic-data-6.csv).

No primeiro instante, ao carregar o conjunto de dados, foi necessario avaliar suas características como, seu esquema de organização, consistência dos dados e a necessidade possíveis correções e adaptações que pudessem contribuir na manipulação e pesquisa. Neste passo foram aplicadas as seguintes mudanças: 
- Nomes de colunas e valores foram traduzidos para o português de modo que ficassem no mesmo idioma da análise;
- Valores nulos na coluna 'idade' foram preenchidos com o valor médio;
- Passageiros foram classificados por idade como Criança, menores de 15 anos, Jovem, entre 15 e 25, Adulto, entre 25 e 65, e Idoso para maiores de 65 anos;
- Valores abreviados na coluna 'embarcou' foram substituidos pelo nome correspondente sem abreviação;
- Passageiros que sobreviveram e não possuiam local de embarque foram removidos do conjunto, considerando que não embarcaram;


Feito isso, foram levantadas perguntas das quais poderiam ser respondidas com o conjunto de dados disponível, são elas:
1. Quantos sobreviveram?
2. Quantos morreram?
3. Quantas crianças sobreviveram?
4. Quantas crianças morreram?
5. Qual a idade média entre os sobreviventes?
6. Qual a idade média entre os que morreram?
7. Qual a relação dos sobreviventes com a classe de ingresso?
8. Qual a relação dos sobreviventes com a classe e sexo?
9. Qual ponto de embarque recebeu mais passageiros? 
10. Qual sexo mais relevante entre os sobreviventes?
11. Qual a relação entre as tarifas e os sobreviventes?
12. Quantos bilhetes únicos existem?
13. Quantos bilhetes duplicados existem?
14. Quais as características dos passageiros com bilhetes duplicados?
15. Qual a relação entre os sobreviventes e os pontos de embarque?

## Perguntas

### 1 e 2, Quantos sobreviveram e quantos morreram? 

Complementares, a primeira e a segunda pergunta foram respondidas ao agrupar os dados por tipo de valor na coluna 'sobreviveu' e apontam que, conforme a tabela e gráfico abaixo, 340 (38,25%) pessoas sobreviveram e outras 549 (61,75%) morreram.

In [ ]:
plot_survivors()

### 3 e 4, Quantas crianças sobreviveram e quantas morreram?

Para as perguntas três e quatro, que também são complementares, foram filtrados os registros que apresentavam o valor 'Criança' na coluna 'categoria_idade' para posteriormente contabilizar os valores de sobreviventes.

In [ ]:
plot_children_balance()

### 5 e 6, Qual a idade média entre os sobreviventes? E entre os que não sobreviveram? 

Os sobreviventes aprensentavam idade média de 28,47 anos e os que não sobreviveram com 30,48. Ambos aprensetaram um pico de densidade de passageiros entre 20 e 40 anos, sendo de sobreviventes levemente superior a 4% e dos não sobreviventes superior a 5%.

In [ ]:
plot_age_dist()

### 7. Qual a relação dos sobreviventes com a classe de ingresso?

A classe que apresentou maior número de sobreviventes foi a primeira com 62,62%, seguida da segunda classe com 52,72% e a terceira classe com 24,24%. Podemos concluir que algum fator, possivelmente, favoreceu os passageiros da primeira classe no acesso aos botes salva vidas.

In [ ]:
plot_surv_by_class()

### 8. Qual o sexo mais relevante entre os sobreviventes?

O sexo mais relevante entre os sobreviventes é o feminino com 67,94%.

In [ ]:
sex_survivors_compare()

In [ ]:
df.query('sobreviveu == "Sim"')['sexo'].value_counts(normalize=True)

### 9. Qual a relação dos sobreviventes com a classe e sexo? 

Ao analisarmos a proporção de sobreviventes por gênero e classe é possível notar como as mulheres da primeira e segunda se destacam em relação a seu grupo. Sobreviveram, 96,74% das mulheres da primeira classe e 92,11% das mulheres da segunda classe, ao passo que a proporção de homens, respectivamente, foi de 36,89% e 15,74%. 
O gênero que apresentou pior proporção foi o masculino na terceira classe com apenas 13,54% de sobreviventes.

In [ ]:
plot_survivors_by_class_sex()

### 10. Qual ponto de embarque recebeu mais passageiros?

In [ ]:
df['embarcou'].value_counts().to_frame()

O ponto de embarque que recebeu mais passageiros foi de Southampton, ao sul do Reino Unido e também ponto de partida do navio.

### 11.

Este é um projeto aberto, não tem como objetivo encontrar respostas definitivas e campos que não foram analisados ser explorados em novas análises e ou revistos.

---

# Consultas e referências

- [Seaborn API documentation](https://seaborn.pydata.org/api.html)
- [Stack Overflow: Seaborn](https://stackoverflow.com/questions/33524694/plotting-with-seaborn)
- [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/)
- [Stack Overflow: Pandas](https://stackoverflow.com/questions/tagged/pandas)
- [Matplotlib documentation](https://matplotlib.org/contents.html)



